In [1]:
import requests
import json
import time
import math
import re
import calendar
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timezone
import yaml
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn.neighbors import NearestNeighbors
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
import os, sys, glob
import kaleido
from PIL import Image
from fpdf import FPDF
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [2]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_f_time=var_time.hour
    var_julian_date=parsed_date.timetuple().tm_yday
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_f_time, var_julian_date, var_weekday, var_weekday_name

In [3]:
def eda(data):
    output=[]
    for col in data.columns:
        duplicatedvalue = data[col].duplicated().sum()
        duplicatedrows = data.duplicated().sum()
        missingvalue = np.sum(pd.isna(data[col]))
        uniquevalue = data[col].nunique()
        datatype = str(data[col].dtype)
        
        output.append([col, duplicatedvalue, duplicatedrows, missingvalue, uniquevalue, datatype])
        
    output = pd.DataFrame(output) 
    output.columns = ['Features', 'Duplicated Values', 'Duplicated Rows', 'Missing Values', 'Unique Values', 'Data Type']
    display(output)

In [4]:
def get_last_value(value):
    last = value
    return last

In [5]:
def perc_on_bar(plot, feature):
    total = len(feature)
    for p in ax.patches:
        percentage = "{:.1f}%".format(100 * p.get_height() / total)
        x = p.get_x() + p.get_width() / 2 - 0.05
        y = p.get_y() + p.get_height()
        ax.annotate(percentage, (x, y), size=12)
    plt.show()

In [37]:
def viz(data):
    fig = go.Figure(data=[go.Candlestick(x=data['UTC_Time'],
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'])])

    fig.update_layout(xaxis_rangeslider_visible=False,
                      title_font_color="blue",
                      title_font_size = 20)
    
    fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])   
    fig.show()

# <font color='red'>Currency Pair</font>

In [10]:
asia = ['EUR_NZD','EUR_AUD','GBP_NZD','GBP_AUD','AUD_USD','AUD_CAD','AUD_CHF','AUD_NZD','NZD_USD','NZD_CHF','NZD_CAD']

currency_pairs = ['EUR_USD','EUR_GBP','EUR_NZD','EUR_AUD','EUR_CHF','EUR_CAD',
                  'GBP_USD','GBP_CHF','GBP_NZD','GBP_AUD','GBP_CAD','AUD_USD',
                  'AUD_CAD','AUD_CHF','AUD_NZD','NZD_USD','NZD_CHF','NZD_CAD',
                  'USD_CAD','USD_CHF','CAD_CHF']

currency_pairs = ["EUR_USD"]


timeframe = "H4"
#D #H1 #H4 M30
# https://developer.oanda.com/rest-live-v20/instrument-df/#CandlestickGranularity
price_char = "M"
#M(midpoint candles) #B(bid candles) #A(ask candles) #BA
price_com = "mid"
#mid #bid #ask


# <font color='red'>Test Configs</font>

In [11]:
Test_Candle = [2108, 8371, 4614, 8207, 2281, 5172, 3342, 121, 8607, 6387, 9303, 3835, 2134, 6930, 6858, 7301, 8099, 985, 5419, 7471, 1457, 8910, 3841, 704, 7308, 8734, 7172, 7385, 1613, 5421, 7061, 1088, 4383, 9123, 166, 9414, 3899, 4915, 988, 5856, 4308, 742, 5213, 6201, 3898, 1346, 9255, 2498, 6305, 4755, 837, 4628, 805, 3896, 5168, 237, 6147, 4220, 3683, 6668, 989, 216, 4455, 3231, 3456, 8141, 629, 8250, 8351, 8648, 2499, 1818, 1253, 22, 3337, 8189, 2423, 5209, 1426, 4332, 3663, 8871, 1620, 8493, 7271, 214, 6443, 97, 2524, 1767, 7839, 3644, 6230, 7605, 4921, 5945, 2162, 3773, 7216, 2936]

### Logging

In [12]:
filename = "BackTest_{}.csv".format(currency_pairs[0])
data = pd.read_csv(filename)

In [13]:
data.shape

(9646, 29)

In [14]:
data.head(2)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,...,Direction,col_1,col_2,col_3,col_4,col_5,SMA_10,SMA_20,F_SMA_10,F_SMA_20
0,2015-08-18,21:00:00,21,230,1,Tuesday,2015-08-18T21:00:00.000000000Z,967,1.10242,1.10391,...,1,-0.00138,0.00011,0.00035,0.00052,0.00004,1.104040,1.105959,-0.000240,-0.002159
1,2015-08-19,01:00:00,1,231,2,Wednesday,2015-08-19T01:00:00.000000000Z,1665,1.10380,1.10526,...,1,-0.00109,0.00037,0.00019,-0.00138,0.00052,1.103502,1.105358,0.001388,-0.000468


In [15]:
data.tail(2)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,...,Direction,col_1,col_2,col_3,col_4,col_5,SMA_10,SMA_20,F_SMA_10,F_SMA_20
9644,2021-11-01,13:00:00,13,305,0,Monday,2021-11-01T13:00:00.000000000Z,21754,1.15775,1.15941,...,1,-0.00145,0.00021,0.00119,-0.00054,-0.00204,1.156968,1.158897,0.002232,0.000303
9645,2021-11-01,17:00:00,17,305,0,Monday,2021-11-01T17:00:00.000000000Z,475,1.15921,1.15949,...,1,-0.00023,0.00005,0.00002,-0.00145,-0.00054,1.157756,1.158197,0.001684,0.001243


# <font color='red'>CANDLE INDEX NUMBER</font>

In [17]:
candle_no =  100 # must be set from test array

# Calculating Stop Loss

In [22]:
STOP_LOSS = data.iloc[candle_no]['ATR_14']
STOP_LOSS

0.0046999999999999

In [23]:
CLOSED_PRICE = data.iloc[candle_no]['Close']
CLOSED_PRICE

1.12112

In [24]:
BUY_SL = CLOSED_PRICE - STOP_LOSS
BUY_SL = BUY_SL.round(5)

In [25]:
SELL_SL = CLOSED_PRICE + STOP_LOSS
SELL_SL = SELL_SL.round(5)

# Modeling

### Feature Selection and Reduction

In [27]:
data = data.drop(columns=['Volume','Weekday','Date','Time',
                          'Weekday_Name','UTC_Time','Direction',
                          'Open', 'High', 'Low', 'Close',
                          'O-H','O-L','O-C','H-L','H-C','L-C',
                          'SMA_10','SMA_20',
                          'f_time','julian_date','ATR_14',
#                          'col_1','col_2','col_3',
                          'col_4','col_5',
#                          'F_SMA_10','F_SMA_20'
                         ])

# Comment out the features that you want to fit into your model.

### Feature Being Fit to the Model

In [28]:
data.head()

,col_1,col_2,col_3,F_SMA_10,F_SMA_20
0,-0.00138,0.00011,0.00035,-0.000240,-0.002159
1,-0.00109,0.00037,0.00019,0.001388,-0.000468
2,-0.00046,0.00196,0.00009,0.001438,0.000273
3,0.00168,0.00396,0.00358,-0.000378,-0.001030
4,-0.00210,0.00128,0.00144,0.001090,0.001248


In [29]:
def find_k_similar_candles(candle_id, dataset, k=4):
    indices=[]
    distances = []
    output = []
    model_knn = NearestNeighbors(metric = 'euclidean', algorithm = 'auto') 
    model_knn.fit(dataset)
    
    #metric = 'euclidean' or 'cosine' or 'manhattan' or 'mahalanobis'
    
    distances, indices = model_knn.kneighbors(dataset.iloc[candle_id,:].values.reshape(1,-1),
                                              n_neighbors = k)

    for i in range(0,len(distances.flatten())):
        if i==0:
            display (pd.DataFrame(data.iloc[candle_id]).transpose())
            #print("Recommendation for {0}:\n".format(eurusd_data.index[candle_id]))
        else:
            #print("{0}: {1}, with distance of {2}".format(i,
            #                                               dataset.index[indices.flatten()[i]],
            #                                               distances.flatten()[i]))
            
            output.append ([dataset.index[indices.flatten()[i]],
                            distances.flatten()[i],
#                           dataset.iloc[indices.flatten()[i]]['O-H'],dataset.iloc[indices.flatten()[i]]['O-L'],dataset.iloc[indices.flatten()[i]]['O-C'],dataset.iloc[indices.flatten()[i]]['H-L'],dataset.iloc[indices.flatten()[i]]['H-C'],dataset.iloc[indices.flatten()[i]]['L-C'],
                           dataset.iloc[indices.flatten()[i]]['col_1'],dataset.iloc[indices.flatten()[i]]['col_2'],dataset.iloc[indices.flatten()[i]]['col_3'],
#                           dataset.iloc[indices.flatten()[i]]['col_4'],dataset.iloc[indices.flatten()[i]]['col_5'],
                           dataset.iloc[indices.flatten()[i]]['F_SMA_10'],dataset.iloc[indices.flatten()[i]]['F_SMA_20'],
                           ])
    
    output = pd.DataFrame(output)
    output.columns = ['Indice','Distance',
#                      'O-H','O-L','O-C','H-L','H-C','L-C',
                      'col_1','col_2','col_3',
#                      'col_4','col_5',
                      'F_SMA_10','F_SMA_20'
                     ]
    display (output)
    
    return indices, distances

### Top 5 Similar Candlesticks

In [30]:
indices, distances = find_k_similar_candles (candle_no,data)

,col_1,col_2,col_3,F_SMA_10,F_SMA_20
100,-0.00136,0.00073,0.00257,-0.000104,0.002025


,Indice,Distance,col_1,col_2,col_3,F_SMA_10,F_SMA_20
0,4548,0.000894,-0.00119,0.00093,0.00197,-0.000662,0.001782
1,7620,0.001019,-0.00058,0.00078,0.00196,0.000050,0.001848
2,4241,0.001051,-0.00072,0.00046,0.00230,0.000430,0.002539


In [31]:
indices = indices[0:1][0]

In [32]:
indices

array([ 100, 4548, 7620, 4241])

### Currnet Market/Candlestick

In [57]:
closed_candle = "currnet_market_data.csv"
data = pd.read_csv(filename)
data = data.iloc[candle_no:candle_no+30]
data.to_csv(closed_candle, header = True, index = False)
#viz(data)

In [56]:
data['candleno'] = range (1, len(data) + 1)
X = data['candleno'].values.reshape(-1, 1)
Y = data['Close'].values.reshape(-1, 1)
linear_regressor = LinearRegression()
linear_regressor.fit(X, Y)
y_pred = linear_regressor.predict(X) 
#plt.scatter(X, Y)
#plt.plot(X, y_pred, color='red')
#plt.show()

In [51]:
print(r2_score(Y, y_pred).round(2)*100, '% Fit') # 1.  Saved Data -> current market , fit
coeficient = (linear_regressor.coef_)

if coeficient > 0:
    print('Action: Bullish')    #  2. Saved Data  -> current market
    print('STOP LOSS:', BUY_SL) #  3. Saved Data  -> current market , buy stoploss
else:
    print('Action: Bearish')     # 2. Saved Data  -> current market
    print('STOP LOSS:', SELL_SL) # 3. Saved Data  -> current market , sell stoploss

1.0 % Fit
Action: Bullish
STOP LOSS: 1.11642


### Recommendations

In [53]:
for indice in indices[1:10]:
    data = pd.read_csv(filename) 
    data = data.iloc[indice:indice+30]
    
    #viz(data)
    
    data['candleno'] = range (1, len(data) + 1)
    X = data['candleno'].values.reshape(-1, 1)
    Y = data['Close'].values.reshape(-1, 1)
    linear_regressor = LinearRegression()
    linear_regressor.fit(X, Y)
    y_pred = linear_regressor.predict(X)
    #plt.scatter(X, Y)
    #plt.plot(X, y_pred, color='red')
    #plt.show()
    
    print(r2_score(Y, y_pred).round(2)*100, '% Fit') #   4,5,6 Saved Data  -> Predicted market --- Fit
    coeficient = (linear_regressor.coef_)
    
    if coeficient > 0:
        print('Action: BUY') #  4,5,6 Saved Data  -> Predicted market

    else:
        print('Action: SELL') #  4,5,6 Saved Data  -> Predicted market




30.0 % Fit
Action: SELL
43.0 % Fit
Action: BUY
80.0 % Fit
Action: SELL


### Save Backtest